# ISMB figures notebook

In [29]:

include("engines/init.jl")
include("engines/data_processing.jl")
include("engines/deep_learning.jl")
include("engines/cross_validation.jl")
outpath, session_id = set_dirs() ;

  Activating project at `~/vae_cox`


In [ ]:
BRCA = MLSurvDataset("Data/TCGA_BRCA_tpm_n1049_btypes_labels_surv.h5")

In [2]:
LAML = MLSurvDataset("Data/LGN_AML_tpm_n300_btypes_labels_surv.h5") 

MLSurvDataset(Float32[0.008600163 0.0 … 0.0 0.0; 0.033423774 0.00432137 … 0.0 0.029383799; … ; 0.4828736 0.0 … 0.0 0.15836251; 0.045322984 0.017033324 … 0.0 0.20682588], ["01H001", "02H003", "02H009", "02H017", "02H026", "02H033", "02H053", "02H066", "03H016", "03H022"  …  "13H186", "14H001", "14H007", "14H012", "14H015", "14H017", "14H019", "14H020", "14H023", "14H038"], ["TSPAN6", "TNMD", "DPM1", "SCYL3", "C1orf112", "FGR", "CFH", "FUCA2", "GCLC", "NFYA"  …  "AP003086.3", "AL109627.1", "AC084851.4", "AC024558.2", "AC108479.4", "AL512357.2", "AL138899.3", "AL669830.1", "AC091135.2", "AL357075.5"], ["lncRNA", "lncRNA", "protein_coding", "lncRNA", "protein_coding", "lncRNA", "protein_coding,retained_intron", "lncRNA", "protein_coding", "protein_coding"  …  "transcribed_processed_pseudogene", "lncRNA", "processed_pseudogene", "protein_coding", "unprocessed_pseudogene", "lncRNA", "protein_coding", "retained_intron", "unprocessed_pseudogene", "protein_coding"], ["Therapy-related myeloid ne

In [86]:

function data_prep(DATA;nfolds = 5, nepochs =1000, dim_redux= 125)
    keep = [occursin("protein_coding", bt) for bt in DATA.biotypes]
    println("nb genes : $(sum(keep))")
    println("nb patients : $(size(DATA.samples)[1])")
    println("% uncensored : $(mean(DATA.surve .!= 0))")
    params_dict = Dict(
            ## run infos 
            "session_id" => session_id, "nfolds" =>5,  "modelid" => "$(bytes2hex(sha256("$(now())"))[1:Int(floor(end/3))])",
            "machine_id"=>strip(read(`hostname`, String)), "device" => "$(device())", "model_title"=>"AECPHDNN",
            ## data infos 
            "dataset" => "BRCA_data(norm=true)", "nsamples" => size(DATA.samples)[1],
            "nsamples_test" => Int(round(size(DATA.samples)[1] / nfolds)), "ngenes" => size(DATA.genes[keep])[1],
            "nsamples_train" => size(DATA.samples)[1] - Int(round(size(DATA.samples)[1] / nfolds)),
            ## optim infos 
            "nepochs" => nepochs, "ae_lr" =>1e-6, "cph_lr" => 1e-5, "ae_wd" => 1e-6, "cph_wd" => 1e-4,
            ## model infos
            "model_type"=> "vaecox", "dim_redux" => dim_redux, "ae_nb_hls" => 2,
            "enc_nb_hl" => 2, "enc_hl_size"=> 128,
            "venc_nb_hl" => 2, "venc_hl_size"=> 128,  "dec_nb_hl" => 2 , "dec_hl_size"=> 128,
            "nb_clinf" => 0, "cph_nb_hl" => 2, "cph_hl_size" => 64, 
            "insize" => size(DATA.genes[keep])[1],
            ## metrics
            "model_cv_complete" => false
        )
    # split train test
    folds = split_train_test(Matrix(DATA.data[:,keep]), DATA.survt, DATA.surve, DATA.samples;nfolds =5)
    fold = folds[1]
    # format input data  
    train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst = format_train_test(fold)

    return train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst, params_dict
end
train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst, params_dict = data_prep(LAML)
# create model 
model = build_vaecox(params_dict)


nb genes : 14996
nb patients : 300
% uncensored : 0.7433333333333333


Dict{String, Any} with 3 entries:
  "cph"  => Chain(Dense(125 => 64, leakyrelu), Dense(64 => 64, leakyrelu), Dens…
  "venc" => VariationalEncoder(Dense(14996 => 128, leakyrelu), Dense(128 => 125…
  "vdec" => Chain(Dense(125 => 128, leakyrelu), Dense(128 => 14996))

In [ ]:

function build_vaecox(params_dict)
    encoder = VariationalEncoder(params_dict["insize"], params_dict["dim_redux"], params_dict["venc_hl_size"]) # chain 
    decoder = Decoder(params_dict["insize"], params_dict["dim_redux"], params_dict["venc_hl_size"]) # chain 
    cph = gpu(Chain(Dense(params_dict["dim_redux"], params_dict["cph_hl_size"], leakyrelu), Dense(params_dict["cph_hl_size"], params_dict["cph_hl_size"],leakyrelu), Dense(params_dict["cph_hl_size"], 1))) # chain 
    return Dict("venc" => encoder,
    "vdec" => decoder,
    "cph" => cph )
end

In [6]:
function VAE_COX_loss(VENC, CPH, X, Y_e, NE_frac;device = gpu)
    mu, log_sigma = VENC(X)
    #z = mu + device(randn(Float32, size(log_sigma))) .* exp.(log_sigma)
    outs = vec(CPH(mu))
    hazard_ratios = exp.(outs)
    log_risk = log.(cumsum(hazard_ratios))
    uncensored_likelihood = outs .- log_risk
    censored_likelihood = uncensored_likelihood .* Y_e'
    #neg_likelihood = - sum(censored_likelihood) / sum(e .== 1)
    neg_likelihood = - sum(censored_likelihood) * NE_frac
    return neg_likelihood
end 

VAE_COX_loss (generic function with 1 method)

In [81]:
function format_train_test(fold; device = gpu)
    nsamples = size(fold["train_x"])[1]
    ordering = collect(1:nsamples)#sortperm(-fold["Y_t_train"])
    train_x = device(Matrix(fold["train_x"][ordering,:]'));
    train_y_t = device(Matrix(fold["Y_t_train"][ordering,:]'));
    train_y_e = device(Matrix(fold["Y_e_train"][ordering,:]'));
    NE_frac_tr = sum(train_y_e .== 1) != 0 ? 1 / sum(train_y_e .== 1) : 0

    nsamples = size(fold["test_x"])[1]
    ordering = collect(1:nsamples)#sortperm(-fold["Y_t_test"])
    test_x = device(Matrix(fold["test_x"][ordering,:]'));
    test_y_t = device(Matrix(fold["Y_t_test"][ordering,:]'));
    test_y_e = device(Matrix(fold["Y_e_test"][ordering,:]'));
    NE_frac_tst = sum(test_y_e .== 1) != 0 ? 1 / sum(test_y_e .== 1) : 0
    return train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst
end 

format_train_test (generic function with 1 method)

In [72]:
function cox_nll_vec(mdl::Chain, X_, Y_e_, NE_frac)
    outs = vec(mdl(X_))
    #outs = vec(mdl.cphdnn(mdl.encoder(X_)))
    hazard_ratios = exp.(outs)
    log_risk = log.(cumsum(hazard_ratios))
    uncensored_likelihood = outs .- log_risk
    censored_likelihood = uncensored_likelihood .* Y_e_'
    #neg_likelihood = - sum(censored_likelihood) / sum(e .== 1)
    neg_likelihood = - sum(censored_likelihood) * NE_frac
    return neg_likelihood
end 

cox_nll_vec (generic function with 6 methods)

In [85]:
train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst, params_dict = data_prep(LAML)
# create model 
#model = build_vaecox(params_dict)
cphdnn = gpu(Chain(Dense(size(train_x)[1], 3040, leakyrelu), Dense(3040, 616, leakyrelu), Dense(616,128, leakyrelu), Dense(128,64, leakyrelu),Dense(64, 1, bias = false)))
opt = Flux.ADAM(1e-4)
wd = params_dict["cph_wd"]
for i in 1:1000
    ps1 = Flux.params(cphdnn)
    gs1 = gradient(ps1) do 
        cox_nll_vec(cphdnn, train_x, train_y_e, NE_frac_tr) + l2_penalty(cphdnn) * wd
        #VAE_COX_loss(model["venc"], model["cph"], train_x, train_y_e, NE_frac_tr) + l2_penalty(model["venc"]) * wd + l2_penalty(model["cph"]) * wd 
    end 
    cphloss = cox_nll_vec(cphdnn, train_x, train_y_e, NE_frac_tr) + l2_penalty(cphdnn) * wd
    #cphloss = round(VAE_COX_loss(model["venc"], model["cph"], train_x, train_y_e, NE_frac_tr) + l2_penalty(model["venc"]) * wd + l2_penalty(model["cph"]) * wd , digits = 3)
    #OUTS_tr = model["cph"](model["venc"](train_x)[1])
    #OUTS_tst =  model["cph"](model["venc"](test_x)[1])
    OUTS_tr = cphdnn(train_x)
    OUTS_tst = cphdnn(test_x)
    cind_tr, cdnt_tr, ddnt_tr, tied_tr  = concordance_index(train_y_t, train_y_e, OUTS_tr)
    cind_test,cdnt_tst, ddnt_tst, tied_tst = concordance_index(test_y_t, test_y_e,OUTS_tst)
    
    if i%100==0 || i == 1
    println("$i TRAIN $cphloss cind: $(round(cind_tr, digits = 3)) \t TEST cind: $(round(cind_test, digits = 3)) [$(Int(cdnt_tst)), $(Int(ddnt_tst)), $(Int(tied_tst))]")
    end 
    Flux.update!(opt,ps1, gs1)
end 

nb genes : 14996
nb patients : 300
% uncensored : 0.7433333333333333


1 TRAIN 5.117351373931716 cind: 0.507 	 TEST cind: 0.518 [824, 768, 0]


100 TRAIN 2.1551450381452866 cind: 0.42 	 TEST cind: 0.41 [652, 940, 0]


200 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


300 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


...

600 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


700 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


800 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


900 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


1000 TRAIN NaN cind: NaN 	 TEST cind: NaN [0, 0, -60]


cox_nll_vec (generic function with 6 methods)

In [ ]:
cphdnn = gpu(Chain(Dense(size(DATA.data[:,keep])[2], 64, leakyrelu), Dense(64, 64,leakyrelu),Dense(64, 1)))

In [ ]:
train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst = format_train_test(fold)
cphdnn(train_x)

In [ ]:
function validate_vaecox(DATA, params_dict;device = gpu)
    nfolds, nepochs, dim_redux = 5, 1000, 125
    keep = [occursin("protein_coding", bt) for bt in DATA.biotypes]
    # split train test
    folds = split_train_test(Matrix(DATA.data[:,keep]), DATA.survt, DATA.surve, DATA.samples;nfolds =5)
    fold = folds[1]

    venc = VariationalEncoder(size(DATA.data[:,keep])[2], 125, 600)
    vdec = Decoder(size(DATA.data[:,keep])[2], 125, 600)
    VAE_opt = Flux.ADAM(1e-4)

    cphdnn = device(Chain(Dense(size(DATA.data[:,keep])[2], 125, leakyrelu), Dense(125, 100,leakyrelu),Dense(100, 1)))
    cphdnn_opt = Flux.ADAM(1e-5)

    train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst = format_train_test(fold)
    for i in 1:nepochs
        ps1 = Flux.params(venc, vdec)
        gs1 = gradient(ps1) do
            VAE_lossf(venc, vdec, train_x)
        end 
        VAE_loss = VAE_lossf(venc, vdec, train_x)
        VAE_cor = round(my_cor(vec(train_x), vec(MyReconstruct(venc, vdec, train_x)[end])),digits = 3)
        VAE_test = round(my_cor(vec(test_x), vec(MyReconstruct(venc, vdec, test_x)[end])),digits = 3)

        ps2 = Flux.params(venc, cphdnn)
        gs2 = gradient(ps2) do 
            cox_nll_vec(cphdnn, train_x, train_y_e, NE_frac_tr)
            #VAE_COX_loss(venc, cphdnn, train_x, train_y_e, NE_frac_tr)
        end 
        CPH_loss = round(cox_nll_vec(cphdnn, train_x, train_y_e, NE_frac_tr), digits = 3) #round(VAE_COX_loss(venc, cphdnn, train_x, train_y_e, NE_frac_tr), digits = 3)
        #mu, log_sigma = venc(train_x)
        #z = mu + device(randn(Float32, size(log_sigma))) .* exp.(log_sigma)
        #OUTS_tr = vec(cphdnn(z))
        OUTS_tr = cphdnn(train_x)
        OUTS_tst =  cphdnn(test_x)
        cind_tr, cdnt_tr, ddnt_tr, tied_tr  = concordance_index(train_y_t, train_y_e, OUTS_tr)
        cind_test,cdnt_tst, ddnt_tst, tied_tst = concordance_index(test_y_t, test_y_e,OUTS_tst)
        #Flux.update!(VAE_opt, ps1, gs1)
        Flux.update!(cphdnn_opt, ps2, gs2)
        if i % 100 == 0 || i == 1
            println("$i TRAIN - VAE-loss-avg: $VAE_loss\tVAE-cor: $VAE_cor CPH-loss: $CPH_loss CPH-cind: $(round(cind_tr,digits=3))")
            println("$i TEST - VAE-loss-avg: $VAE_test \tVAE-cor: CPH-loss: CPH-cind: $(round(cind_test,digits=3)) [$(Int(cdnt_tst)), $(Int(ddnt_tst)), $(Int(tied_tst))]")

        end
    end
    return cphdnn
end 

In [ ]:
cdnn = validate_vaecox(DATA, params_dict)

In [ ]:
cdnn(DATA.data[])

In [88]:
train_x, train_y_t, train_y_e, NE_frac_tr, test_x, test_y_t, test_y_e, NE_frac_tst, params_dict = data_prep(LAML)
# create model 
model = build_aecox(params_dict)
for iter in 1:params_dict["nepochs"]
    ps1 = Flux.params(model["cph"].model, model["enc"])
    gs1 = gradient(ps1) do
        model["cph"].lossf(model["cph"],model["enc"], train_x, train_y_e, NE_frac_tr, params_dict["cph_wd"])
    end 
    ## gradient Auto-Encoder 
    ps2 = Flux.params(model["ae"].net)
    gs2 = gradient(ps2) do
        model["ae"].lossf(model["ae"], train_x, train_x, weight_decay = params_dict["ae_wd"])
    end
    Flux.update!(model["cph"].opt, ps1, gs1)
    #Flux.update!(model["ae"].opt, ps2, gs2)

    ######
    OUTS_tr = vec(model["cph"].model(model["enc"](train_x)))
    ae_loss = model["ae"].lossf(model["ae"], train_x, train_x, weight_decay = params_dict["ae_wd"])
    ae_cor =  round(my_cor(vec(train_x), vec(model["ae"].net(train_x))),digits = 3)
    cph_loss = model["cph"].lossf(model["cph"],model["enc"](train_x), train_y_e, NE_frac_tr, params_dict["cph_wd"])
    ae_loss_test = round(model["ae"].lossf(model["ae"], test_x, test_x, weight_decay = params_dict["ae_wd"]), digits = 3)
    ae_cor_test = round(my_cor(vec(test_x), vec(model["ae"].net(test_x))), digits= 3)
    cph_loss_test = round(model["cph"].lossf(model["cph"],model["enc"](test_x), test_y_e, NE_frac_tst, params_dict["cph_wd"]), digits= 3)
                    
    OUTS_tst =  vec(model["cph"].model(model["enc"](test_x)))
            
    cind_tr, cdnt_tr, ddnt_tr, tied_tr  = concordance_index(train_y_t, train_y_e, OUTS_tr)
    cind_test,cdnt_tst, ddnt_tst, tied_tst = concordance_index(test_y_t, test_y_e,OUTS_tst)
    if iter % 100 == 0  || iter == 1     
        println("FOLD $iter\t TRAIN AE-loss $(round(ae_loss,digits =3)) \t AE-cor: $(round(ae_cor, digits = 3))\t cph-loss-avg: $(round(cph_loss / params_dict["nsamples_train"],digits =6)) \t cph-cind: $(round(cind_tr,digits =3))")
        println("\t\tTEST AE-loss $(round(ae_loss_test,digits =3)) \t AE-cor: $(round(ae_cor_test, digits = 3))\t cph-loss-avg: $(round(cph_loss_test / params_dict["nsamples_test"],digits =6)) \t cph-cind: $(round(cind_test,digits =3)) [$(Int(cdnt_tst)), $(Int(ddnt_tst)), $(Int(tied_tst))]")
    end
end


nb genes : 14996
nb patients : 300
% uncensored : 0.7433333333333333


FOLD 1	 TRAIN AE-loss 1.0 	 AE-cor: 0.01	 cph-loss-avg: 0.018436 	 cph-cind: 0.498
		TEST AE-loss 0.99 	 AE-cor: 0.013	 cph-loss-avg: 0.051483 	 cph-cind: 0.479 [787, 855, 0]


FOLD 100	 TRAIN AE-loss 0.786 	 AE-cor: 0.247	 cph-loss-avg: 0.017247 	 cph-cind: 0.648
		TEST AE-loss 0.78 	 AE-cor: 0.248	 cph-loss-avg: 0.050717 	 cph-cind: 0.612 [1005, 637, 0]


FOLD 200	 TRAIN AE-loss 0.52 	 AE-cor: 0.516	 cph-loss-avg: 0.017099 	 cph-cind: 0.651
		TEST AE-loss 0.52 	 AE-cor: 0.514	 cph-loss-avg: 0.050767 	 cph-cind: 0.611 [1003, 639, 0]


FOLD 300	 TRAIN AE-loss 0.297 	 AE-cor: 0.73	 cph-loss-avg: 0.017073 	 cph-cind: 0.652
		TEST AE-loss 0.302 	 AE-cor: 0.724	 cph-loss-avg: 0.050817 	 cph-cind: 0.622 [1022, 620, 0]


...

FOLD 600	 TRAIN AE-loss 0.065 	 AE-cor: 0.949	 cph-loss-avg: 0.017042 	 cph-cind: 0.655
		TEST AE-loss 0.075 	 AE-cor: 0.94	 cph-loss-avg: 0.050917 	 cph-cind: 0.627 [1029, 613, 0]


FOLD 700	 TRAIN AE-loss 0.056 	 AE-cor: 0.957	 cph-loss-avg: 0.017045 	 cph-cind: 0.656
		TEST AE-loss 0.066 	 AE-cor: 0.948	 cph-loss-avg: 0.051083 	 cph-cind: 0.629 [1033, 609, 0]


FOLD 800	 TRAIN AE-loss 0.054 	 AE-cor: 0.96	 cph-loss-avg: 0.017039 	 cph-cind: 0.657
		TEST AE-loss 0.064 	 AE-cor: 0.951	 cph-loss-avg: 0.05095 	 cph-cind: 0.63 [1035, 607, 0]


FOLD 900	 TRAIN AE-loss 0.053 	 AE-cor: 0.961	 cph-loss-avg: 0.017038 	 cph-cind: 0.657
		TEST AE-loss 0.063 	 AE-cor: 0.951	 cph-loss-avg: 0.051 	 cph-cind: 0.629 [1032, 610, 0]


FOLD 1000	 TRAIN AE-loss 0.052 	 AE-cor: 0.961	 cph-loss-avg: 0.017038 	 cph-cind: 0.658
		TEST AE-loss 0.062 	 AE-cor: 0.951	 cph-loss-avg: 0.051 	 cph-cind: 0.629 [1033, 609, 0]


In [70]:
model["cph"].model

Chain(
  Dense(125 => 64, leakyrelu),          # 8_064 parameters
  Dense(64 => 1, σ; bias=false),        # 64 parameters
)                   # Total: 3 arrays, 8_128 parameters, 464 bytes.

In [ ]:
params_dict["cph_tst_c_ind"] = concordance_index(yt_test, ye_test, -1 .* cph_outs_test)[1]
params_dict["cph_train_c_ind"] = concordance_index(yt_train, ye_train, -1 .* cph_outs_train)[1]
params_dict["ae_tst_corr"] = my_cor(ae_outs_test, x_test)
params_dict["ae_train_corr"] = my_cor(ae_outs_train, x_train)

params_dict["model_cv_complete"] = true
bson("RES/$model_params_path/params.bson",params_dict)